## **Notebook 08: Working with the Keras API**
### CBE 512. Machine Learning in Chemical Science and Engineering.

&#169; Princeton University

# **Simple Demonstrations in Keras**

In this notebook, we will examine how to build neural network regression models using [Keras](https://keras.io/about/). Keras itself makes prolific use of tools/objects/algorithms made available through [TensorFlow](https://www.tensorflow.org/guide/basics). These are available through `import` following setup of an appropriate python environment.

Run the following cell to get access to all the things needed for this notebook.

In [ ]:
# modules needed for this notebook
import numpy  as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import pydot
import graphviz
import sklearn.metrics as sklm
from tensorflow       import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso

## *How to build Sequential models*

The `Seqeuntial` model class from Keras provides a straightforward approach to building neural networks. The simplicity of implementation also necessitates some restrictions on the architecture complexity. Essentially, if you want the "standard picture" of a neural network, you can achieve that with a `Sequential` model. This will yield a densely connected, feed-forward neural network with a single input layer and a single output layer.  

Let's take a look at how to do that below.

In [ ]:
# Create Model Container
model = keras.Sequential(name="myFirstModel")

# Define Layers
inputLayer= keras.Input(shape=(4,))
layer1= layers.Dense(10,activation='relu',name="myFirstLayer")
layer2= layers.Dense(8,activation='tanh',name="oldNewsLayer")
output= layers.Dense(1,activation=None,name="outputLayer")

# Add layers to model
model.add(inputLayer)
model.add(layer1)
model.add(layer2)
model.add(output)

# Admire Model
model.summary()

## *How to build models with Functional API*

The `Functional` API allows users to create more flexible models/complex architectures that the `Sequential` API. For the majority of tasks, there may be no need to utilize a `Functional` API, but some more interesting ideas and model strategies can only be achieved using this approach. One reason that you may utilize the `Functional` API is if you have inputs that have disparate structures but still characterize your system. For example, you may have a microscopy image along with metadata on the sample; these are two very different data structures. The former is amenable to image-processing techniques (like a CNN) while the latter can be simply represented as a vector input. With the `Functional` API you can combine both inputs and let separate parts of your architecture process them independently before pooling the information.

To start, let's recreate the network we made with `Sequential`.

In [ ]:
# Define Layers
inputLayer = keras.Input(shape=(4,))
layer1= layers.Dense(10,activation='relu',name="myFirstLayer")
layer2= layers.Dense(8,activation='tanh',name="oldNewsLayer")
output= layers.Dense(1,activation=None,name="output")

# Connect layers using "layer calls"
# we want to achieve
# inputLayer --> layer1 --> layer2 --> outputs
x = layer1(inputLayer)
x = layer2(x)
outputs = output(x)

# Build model from inputs/outputs
model = keras.Model(inputs=inputLayer,outputs=outputs,\
        name="mySecondModel")

# Admire Model
model.summary()
keras.utils.plot_model(model,"model.png",dpi=100,show_shapes=True,show_layer_activations=True)

Now, let's spice it up a little bit and create a split architecture. As a motivating premise, we can consider [this work](https://www.biorxiv.org/content/10.1101/2023.06.06.543884v1.abstract) studying properties of intrinsic disordered proteins. In this paper, authors use a 30-dimensional input vector, 20 of which correspond to the composition of amino acids within the polypeptide and 10 of which relate to aspects of the sequence and its patterning. Suppose we want to treat these types of data separately initially for a learning task. We could do that using an architecture like the following:

In [ ]:
# Define Layers
inputLayer_comp = keras.Input(shape=(20,),name="composition")
inputLayer_seq  = keras.Input(shape=(10,),name="sequence stuff")
layer1_comp= layers.Dense(10,activation='relu',name="c1")
layer1_seq= layers.Dense(10,activation='relu',name="s1")
layer2_comp= layers.Dense(1,activation=None,name="c2")
layer2_seq= layers.Dense(5,activation='relu',name="s2")
layer3_seq= layers.Dense(1,activation=None,name="s3")
output= layers.Dense(1,activation=None,name="output")

# Connect layers using "layer calls"
# we want to achieve
# inputLayer --> layer1 --> layer2 --> outputs
xs = layer3_seq(layer2_seq(layer1_seq(inputLayer_seq)))
xc = layer2_comp(layer1_comp(inputLayer_comp))
added = layers.concatenate([xs,xc])
outputs = output(added)

# Build model from inputs/outputs
model = keras.Model(inputs=[inputLayer_comp,inputLayer_seq],outputs=outputs,\
        name="mySplitModel")

# Admire Model
model.summary()
keras.utils.plot_model(model,"model.png",show_shapes=True)

## **Training a model in Keras**

Now that we know how to build a model in Keras, we can see how easy it is to train/test one as well. For this, we are going to utilize some real data that you should have increasing familiarity with. This is the "Solubility" dataset that we have been exploring in class/on homeworks.

The following cell will just retrieve the data and format it into relevant objects.

In [ ]:
# Read in and inspect some of the data to gain some familiarity for what is in
# the file. The rest of the code extracts some possible features and stores them
# in an array X; the labels, which correspond to solubility values, are in y
url_for_data    = "https://raw.githubusercontent.com/webbtheosim/CBE512-MLinChmSciEng/main/data/solubility-regression-cbe512.csv"
data = pd.read_csv(
    url_for_data
)
i0          = list(data.columns).index("MolWt")
allFeatures = data.columns[i0:-1]
outLabel    = 'Solubility'
X = np.array(data[allFeatures])
nFeatures = X.shape[1]
y = np.array(data[outLabel]).reshape([-1,1])
print(X.shape,y.shape)
print(X[0])
print(y[0])
data.head()

The first thing we want to do is do some preprocessing of the data. For this, we can use some of the tools from scikit-learn to get different transformations. Before we do this, I want to get a quick view of the data. This may not always be possible, but with 17 features, we can do it. The next couple of cells are not essential for anything with Keras but are for demonstration purposes.

In [ ]:
features_start_at = list(data.columns).index("MolWt")
feature_names = data.columns[features_start_at:]

# code for pair correlations
subset = [n for n in feature_names if np.random.random()<0.34]
if outLabel not in subset:
  subset.append(outLabel)
sns.pairplot(data[subset],hue=outLabel)
plt.show()

In [ ]:
# code for looking at pair correlations with Solubility
num_cols = 3
num_rows = int( np.ceil(len(feature_names)/ num_cols))
fig, axs = plt.subplots(nrows=num_rows,ncols=num_cols,sharey=True,figsize=(12,12))
axs = axs.flatten()
for i,n in enumerate(feature_names):
  ax = axs[i]
  ax.scatter(data[n], data.Solubility, s=6, alpha=0.4)
  if i % num_cols == 0:
    ax.set_ylabel("Solubility")
  ax.set_xlabel(n)
  plt.tight_layout()
plt.show()

### Creating a train-test split

OK. We have seen the data. There are some correlated features. We are not going to worry about this significantly right now, but we should be mindful of that were we wanting to create the best model that we could. Following a minimal set of "best practices," let's create a test-train split and get on with the model training.

In [ ]:
# create the split
(X_train, X_test, y_train, y_test) \
   = train_test_split(X,y,test_size = 0.2,shuffle=True)

# Perform data transformations
inScaler = StandardScaler() # scaler for features
outScaler= StandardScaler() # scaler for labels
inScaler.fit(X_train)
outScaler.fit(y_train)
Xsc_train = inScaler.transform(X_train)  # these are the scaled features for training
ysc_train = outScaler.transform(y_train) # these are the scaled labels for training
Xsc_test = inScaler.transform(X_test)  # these are the scaled features for training
ysc_test = outScaler.transform(y_test) # these are the scaled labels for training

# check the data distribution
bins    = np.arange(-3,3.1,0.2)
fig, ax = plt.subplots(figsize=(5,5),sharey=True)
ax.hist(ysc_train,bins=bins,histtype='bar',align='mid',\
        label='train',alpha = 0.4,edgecolor='k')
ax.hist(ysc_test,bins=bins,histtype='bar',align='mid',\
        label='test',alpha = 0.4,edgecolor='k')
ax.legend()
plt.show()

## **Build and train a model**

All righty then. Let's whip up a model and train that sucker. Since we are not dealing with anything too fancy, we will roll with the `Sequential` API. Feel free to use `Functional` if you prefer and want that practice.

In [ ]:
nFeatures = X_train.shape[1] # dimensionality of our input vecto
model  = keras.Sequential () # this initializes our simple model, but it doesn't have anhything in it!
hidden1= layers.Dense(20,activation="relu") # here we create 20-neuron layer with relu activation
hidden2= layers.Dense(5,activation="relu")  # this is a 5-neuron layer, again with relu
out    = layers.Dense(1)    # we will only have one output, activation=None means linear/identity
model.add(hidden1)
model.add(hidden2)
model.add(out)
model.build((None,nFeatures)) # this last line specifies the input shape; there are lots of ways to do this
model.summary()
keras.utils.plot_model(model,"model.png",dpi=100,show_shapes=True)

In [ ]:
# set up model optimization specifications through compile
the_optimizer = tf.keras.optimizers.SGD(\
      learning_rate= 0.005, momentum=0.0,nesterov=False,name='SGD')
the_loss      = tf.keras.losses.MeanSquaredError(reduction="sum_over_batch_size",name="MSE")
model.compile(optimizer=the_optimizer,loss=the_loss)

In [ ]:
# now actually do the training
hist = model.fit(x=Xsc_train,y=ysc_train,\
          batch_size=32,
          epochs=100,
          validation_split = 0.1,
          shuffle=True)

### Inspecting model training

By following the output from above, we can get some sense as to how effective our model is learning. However, it may be informative to more directly inspect this in the form of a *training curve*.

In [ ]:
print(hist.history.keys())
loss = hist.history['loss']
val_loss = hist.history['val_loss']
plt.plot(loss,'-k',linewidth=2)
plt.plot(val_loss,':r',linewidth=2)
ax = plt.gca()
ax.set_ylim([0,1])
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")
plt.show()

### Making predictions
Based on our training curve, it looks like we have a pretty fair model. Let's see how it performs on our held-out test set by using the `predict` method.

In [ ]:
ysc_pred = model.predict(Xsc_test)
ypred   = outScaler.inverse_transform(ysc_pred)
plt.plot(y_test,ypred,".")
linearFit = LinearRegression().fit(y_test,ypred)
r2 = sklm.r2_score(y_test,ypred)
mae= sklm.mean_absolute_error(y_test,ypred)
mse=sklm.mean_squared_error(y_test,ypred)
print("r2 = {:>5.2f}, mae = {:>5.2f}, mse = {:>5.2f}".format(r2,mae,mse))
xline = np.array([[-14],[4]])
yline = linearFit.predict(xline)
plt.plot(xline,yline,'-r')
plt.plot(xline,xline,':k')
plt.show()
